# Accenture model evaluation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import re
from dateutil.parser import parse
import string
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
import missingno as msno
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
import scipy
import researchpy as rp
from matplotlib.offsetbox import AnchoredText
import pacmap
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import plotly.express as px
import pickle
import pmdarima as pm
from ipywidgets import Button, HBox, VBox
from ipywidgets import interactive
import ipywidgets
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import plotly.express as px
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split 
import xgboost as xgb
sns.set_style("darkgrid")
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 100

In [2]:
# Select first of vector of atollos cases (0) and append into DF:
def inicio_perdida_tph(DF, variable):
       first_case_atollo = [0]
       for x in range(1, len(DF)):
              if((DF.loc[DF.index[x], variable] == 1) & (DF.loc[DF.index[x-1], variable] == 0)):
                     first_case_atollo.append(1)
              else:
                     first_case_atollo.append(0)
       return first_case_atollo
       
# Select last of vector of atollos cases (0) and append into DF:
def final_perdida_tph(DF, variable):
       last_case_atollo = []
       for x in range((len(DF)-1)):
              if((DF.loc[DF.index[x], variable] == 1) & (DF.loc[DF.index[x+1], variable] == 0)):
                     last_case_atollo.append(1)
              else:
                     last_case_atollo.append(0)
       last_case_atollo.append(1)              
       return last_case_atollo

# Create function Atollo_mod: 0 when initial atollo; 2 when not initial atollo; 1 when no atollo:
def loss_tph(DF,variable):
    
    DF['start loss tph']=inicio_perdida_tph(DF=DF, variable=variable)
    DF['end loss tph']=final_perdida_tph(DF=DF, variable=variable)
    #Create Atollo_mod:
    variable_mod = str(variable+'_mod')    
    DF[variable_mod] =  DF[variable].replace(1,2)   
    DF.loc[DF['start loss tph']==1,variable_mod]=1
    return DF

In [23]:
# Se carga clasificador de mineralogia
kmeans = pickle.load(open('..//..//models//pickles//Modelo update Accenture//Clasificador_dispatch_n_4.pkl', 'rb'))
#kmeans.feature_names_in_
# Se carga la curva de celda de carga v/s granulometria (dado un subcontexto)
dicK = pickle.load(open('..//..//models//pickles//Modelo update Accenture//cc_recommendations_prod.pkl', 'rb'))

# Recomendación 
def recommendationcc(granulometria,edad_sag,cluster):
 
    # Generación del subcontexto
    subcontexto = str(int(edad_sag)) + str(int(cluster))
   
    # Obtención de la recomendación
    consejo=dicK['s'+subcontexto].loc[granulometria]["cc"]
  
    return consejo #pd.DataFrame({"recommendation":list(consejo)})

In [9]:
# read
tags=pd.read_excel("../../data/meta data/tags relevantes.xlsx")
tags_cc=tags.tag.to_list()
dic={}
for i,j in zip(tags.tag,tags.description):
    dic[i]=j

In [10]:
# Se lee csv
liners_age=pd.read_csv('../../data/processed data/liners_age_30September2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')
# Se lee csv
cleaned=pd.read_csv('../../data/processed data/cleaned_28September2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')
cleaned.sort_index(inplace=True)
cleaned.rename(columns=dic,inplace=True)
df_cleaned_age=cleaned.join(liners_age).dropna()
df_cleaned_age["dif_TPH_HH"]=df_cleaned_age["HH TPH"]-df_cleaned_age["TPH"]
df_cleaned_age["dif_CC_HH"]=df_cleaned_age["HH charge cell"]-df_cleaned_age["charge cell"]
df_cleaned_age["dif_CC_LL"]=df_cleaned_age["charge cell"]-df_cleaned_age["LL charge cell"]
df_cleaned_age["loss of TPH"]=df_cleaned_age["dif_TPH_HH"].apply(lambda x: 1 if x>100 else 0)

In [16]:
# Predict cluster
df_cleaned_age["cluster"]=kmeans.predict(df_cleaned_age[['crusher index',
                                              'sag power index',
                                               'ball work index',
                                              'chalcopyrite law',
                                              'pyrite law',
                                              'chalcocite law',
                                              'bornite law',
                                              'covelin law']])

In [24]:
# Recomendación 
Rec_HH_CC=df_cleaned_age.apply(lambda row: recommendationcc(
                                              row["granulometry"],
                                            row["Edad"],
                                            row["cluster"]
                                                    ),axis=1)

# Columna de recomendación
df_cleaned_age["recommendation"]=Rec_HH_CC

In [25]:
# Información de perdida de TPH en los proximos 5 minutos
for i in range(1,6):
    df_cleaned_age[f"loss of TPH_{i}"]=df_cleaned_age["loss of TPH"].shift(i)

df_cleaned_age["greater than HH CC"]=df_cleaned_age["dif_CC_HH"].apply(lambda x: 0 if x>1 else 1)
for i in range(1,6):
    df_cleaned_age[f"greater than HH CC_{i}"]=df_cleaned_age["greater than HH CC"].shift(i)

df_cleaned_age["less than LL CC"]=df_cleaned_age["dif_CC_LL"].apply(lambda x: 0 if x>1 else 1)
for i in range(1,6):
    df_cleaned_age[f"less than LL CC_{i}"]=df_cleaned_age["less than LL CC"].shift(i)

df_cleaned_age["greater than Rec HH CC"]=df_cleaned_age.apply(lambda x: 1 if x["recommendation"]<=x["charge cell"] else 0,axis=1)
for i in range(1,6):
    df_cleaned_age[f"greater than Rec HH CC_{i}"]=df_cleaned_age["greater than Rec HH CC"].shift(i)

df_cleaned_age["Rec HH CC less than HH CC"]=df_cleaned_age.apply(lambda x: 1 if x["recommendation"]<x["HH charge cell"] else 0,axis=1)
for i in range(1,6):
    df_cleaned_age[f"Rec HH CC less than HH CC_{i}"]=df_cleaned_age["Rec HH CC less than HH CC"].shift(i)

In [26]:
df_cleaned_age.dropna(inplace=True)
df_cleaned_age["loss of TPH"]=df_cleaned_age.apply(lambda x: 1 if (x['loss of TPH']==1) or (x['loss of TPH_1']==1) or (x['loss of TPH_2']==1) or (x['loss of TPH_3']==1) or (x['loss of TPH_4']==1) or (x['loss of TPH_5']==1) else 0,axis=1)
#df_cleaned_age["greater than HH CC"]=df_cleaned_age.apply(lambda x: 1 if (x['greater than HH CC_1']==1) or (x['greater than HH CC_2']==1) or (x['greater than HH CC_3']==1) or (x['greater than HH CC_4']==1) or (x['greater than HH CC_5']==1) else 0,axis=1)
#df_cleaned_age["less than LL CC"]=df_cleaned_age.apply(lambda x: 1 if (x['less than LL CC_1']==1) or (x['less than LL CC_2']==1) or (x['less than LL CC_3']==1) or (x['less than LL CC_4']==1) or (x['less than LL CC_5']==1) else 0,axis=1)
#df_cleaned_age["greater than Rec HH CC"]=df_cleaned_age.apply(lambda x: 1 if (x['greater than Rec HH CC_1']==1) or (x['greater than Rec HH CC_2']==1) or (x['greater than Rec HH CC_3']==1) or (x['greater than Rec HH CC_4']==1) or (x['greater than Rec HH CC_5']==1) else 0,axis=1)
#df_cleaned_age["Rec HH CC less than HH CC"]=df_cleaned_age.apply(lambda x: 1 if (x['Rec HH CC less than HH CC_1']==1) or (x['Rec HH CC less than HH CC_2']==1) or (x['Rec HH CC less than HH CC_3']==1) or (x['Rec HH CC less than HH CC_4']==1) or (x['Rec HH CC less than HH CC_5']==1) else 0,axis=1)
df_cleaned_age=loss_tph(df_cleaned_age,"loss of TPH")
df_cleaned_age.head(1)

,sag mill expert system,Limitado por Límite Alto de Tonelaje,Optimizando tonelaje,Limitado por Peso,Limitado por Corriente Bobina,Limitado por torque espesador 2,Limitado por torque espesador 3,Limitado por Potencia Bolas 1,Limitado por Potencia Bolas 2,Limitado por Pebbles,Limitado por Corriente harnero,Limitado por Potencia SAG,Limitado por limite bajo de tonelaje,Limitado por Temperatura bobina,Limitado por presión espesador 2,Limitado por presión espesador 3,pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,sag power index,ball work index,bornite law,torque,charge cell,power,TPH,solid percentage,water,speed,solid,LL charge cell,HH charge cell,HH TPH,stator current,granulometry,Edad,dif_TPH_HH,dif_CC_HH,dif_CC_LL,loss of TPH,cluster,recommendation,loss of TPH_1,loss of TPH_2,loss of TPH_3,loss of TPH_4,loss of TPH_5,greater than HH CC,greater than HH CC_1,greater than HH CC_2,greater than HH CC_3,greater than HH CC_4,greater than HH CC_5,less than LL CC,less than LL CC_1,less than LL CC_2,less than LL CC_3,less than LL CC_4,less than LL CC_5,greater than Rec HH CC,greater than Rec HH CC_1,greater than Rec HH CC_2,greater than Rec HH CC_3,greater than Rec HH CC_4,greater than Rec HH CC_5,Rec HH CC less than HH CC,Rec HH CC less than HH CC_1,Rec HH CC less than HH CC_2,Rec HH CC less than HH CC_3,Rec HH CC less than HH CC_4,Rec HH CC less than HH CC_5,start loss tph,end loss tph,loss of TPH_mod
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-10 04:05:00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,1.76,0.5,0.02,19.7649,98.5569,15.4504,0.05,64.384706,809.23733,22008.766259,4317.030837,73.888038,1539.203093,9.063024,73.155012,770.0,850.0,3950.0,3755.398344,48.829422,1.0,-367.030837,40.76267,39.23733,0,2,852.364266,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0


In [27]:
df_full=df_cleaned_age.copy()

In [28]:
list_df=[]
for i in df_full[df_full["start loss tph"]==1].index:
    try:
        #i="2022-02-27 12:31:00"
        inicio_perdida=i
        test=df_full[inicio_perdida:inicio_perdida+timedelta(hours=5)]
        fin_perdida=min(test[(test["end loss tph"]==1)].index)+timedelta(minutes=1)
        inicio_evento=inicio_perdida-timedelta(minutes=5)
        data=df_full.copy()[inicio_evento:fin_perdida]
        #display(data)
        dic_df={"inicio evento":inicio_evento,"inicio perdida":inicio_perdida,"fin perdida":fin_perdida,"variación HH TPH":data["HH TPH"].std()!=0,"Actua sistema de control":any(data["greater than HH CC"]==1),
        "Supera recomendación del modelo":any(data["greater than Rec HH CC"]==1),"Recomendación menor a HH CC":any(data["Rec HH CC less than HH CC"]==1),"CC menor a LL CC":any(data["less than LL CC"]==1),
        "Causalidad":any(data[data["greater than Rec HH CC"]==1].index<=inicio_perdida)}
        list_df.append(pd.DataFrame(dic_df,index=[1]))  
    except:
        #print(i)
        pass

In [29]:
df_tph_loss=pd.concat(list_df).reset_index(drop=True)
df_tph_loss.head()

,inicio evento,inicio perdida,fin perdida,variación HH TPH,Actua sistema de control,Supera recomendación del modelo,Recomendación menor a HH CC,CC menor a LL CC,Causalidad
0,2020-01-10 05:23:00,2020-01-10 05:28:00,2020-01-10 05:35:00,True,False,False,False,False,False
1,2020-01-10 07:05:00,2020-01-10 07:10:00,2020-01-10 08:41:00,False,True,True,True,False,True
2,2020-01-10 09:05:00,2020-01-10 09:10:00,2020-01-10 10:16:00,False,True,True,True,False,False
3,2020-01-10 10:15:00,2020-01-10 10:20:00,2020-01-10 11:08:00,False,True,True,True,False,False
4,2020-01-14 03:54:00,2020-01-14 03:59:00,2020-01-14 04:31:00,False,True,True,False,False,False


# Perdida de TPH por superar recomendación [+]

In [30]:
evento1=df_tph_loss[(df_tph_loss["Actua sistema de control"]==False) & (df_tph_loss["variación HH TPH"]==False) & (df_tph_loss["CC menor a LL CC"]==False) & (df_tph_loss["Recomendación menor a HH CC"]==True) \
    & (df_tph_loss["Supera recomendación del modelo"]==True)\
    & (df_tph_loss["Causalidad"]==True)].reset_index(drop=True)#.sample(1)
evento1

,inicio evento,inicio perdida,fin perdida,variación HH TPH,Actua sistema de control,Supera recomendación del modelo,Recomendación menor a HH CC,CC menor a LL CC,Causalidad
0,2020-03-03 09:33:00,2020-03-03 09:38:00,2020-03-03 10:17:00,False,False,True,True,False,True
1,2020-03-05 11:24:00,2020-03-05 11:29:00,2020-03-05 11:44:00,False,False,True,True,False,True
2,2020-03-05 15:41:00,2020-03-05 15:46:00,2020-03-05 15:52:00,False,False,True,True,False,True
3,2020-03-10 02:08:00,2020-03-10 02:13:00,2020-03-10 02:19:00,False,False,True,True,False,True
4,2020-03-10 02:19:00,2020-03-10 02:24:00,2020-03-10 02:32:00,False,False,True,True,False,True
...,...,...,...,...,...,...,...,...,...
425,2022-02-27 04:51:00,2022-02-27 04:56:00,2022-02-27 05:21:00,False,False,True,True,False,True
426,2022-02-27 05:29:00,2022-02-27 05:34:00,2022-02-27 05:40:00,False,False,True,True,False,True
427,2022-02-27 05:46:00,2022-02-27 05:51:00,2022-02-27 06:48:00,False,False,True,True,False,True
428,2022-02-27 12:45:00,2022-02-27 12:50:00,2022-02-27 12:58:00,False,False,True,True,False,True


In [31]:
list_loss_tph=[]
for i in range(len(evento1)):
    data=df_full[evento1.iloc[i]["inicio perdida"]:evento1.iloc[i]["fin perdida"]]
    list_loss_tph.append(data["dif_TPH_HH"].sum())
tph1=sum(list_loss_tph)
tph1   

803092.509763787

# Perdida de TPH sin superar recomendación [-]

In [32]:
evento2=df_tph_loss[(df_tph_loss["Actua sistema de control"]==False) & (df_tph_loss["variación HH TPH"]==False) & (df_tph_loss["CC menor a LL CC"]==False) & (df_tph_loss["Recomendación menor a HH CC"]==False)\
     & (df_tph_loss["Supera recomendación del modelo"]==False)\
    ].reset_index(drop=True)
evento2

,inicio evento,inicio perdida,fin perdida,variación HH TPH,Actua sistema de control,Supera recomendación del modelo,Recomendación menor a HH CC,CC menor a LL CC,Causalidad
0,2020-02-05 10:33:00,2020-02-05 10:38:00,2020-02-05 10:44:00,False,False,False,False,False,False
1,2020-02-05 11:30:00,2020-02-05 11:35:00,2020-02-05 11:45:00,False,False,False,False,False,False
2,2020-02-06 09:04:00,2020-02-06 09:09:00,2020-02-06 09:41:00,False,False,False,False,False,False
3,2020-02-06 11:54:00,2020-02-06 11:59:00,2020-02-06 12:07:00,False,False,False,False,False,False
4,2020-02-06 12:57:00,2020-02-06 13:02:00,2020-02-06 13:08:00,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
567,2022-01-15 12:37:00,2022-01-15 12:42:00,2022-01-15 12:50:00,False,False,False,False,False,False
568,2022-01-15 19:29:00,2022-01-15 19:34:00,2022-01-15 19:43:00,False,False,False,False,False,False
569,2022-01-15 21:45:00,2022-01-15 21:50:00,2022-01-15 21:56:00,False,False,False,False,False,False
570,2022-01-16 02:06:00,2022-01-16 02:11:00,2022-01-16 02:20:00,False,False,False,False,False,False


In [33]:
list_loss_tph=[]
for i in range(len(evento2)):
    data=df_full[evento2.iloc[i]["inicio perdida"]:evento2.iloc[i]["fin perdida"]]
    list_loss_tph.append(data["dif_TPH_HH"].sum())
tph2=sum(list_loss_tph)
tph2  

1197166.5202752773

# Costo de oportunidad TPH --> se supera recomendación pero aun así no hay caida de TPH [-]

In [34]:
# Desfase de perdida de TPH para eliminar las causas de las perdidas
df_cost=df_full.copy()
for i in range(1,6):
    df_cost[f"loss of TPH_{i}"]=df_cost["loss of TPH"].shift(-i)

df_cost["loss of TPH"]=df_cost.apply(lambda x: 1 if (x['loss of TPH']==1) or (x['loss of TPH_1']==1) or \
(x['loss of TPH_2']==1) or (x['loss of TPH_3']==1) or (x['loss of TPH_4']==1) or (x['loss of TPH_5']==1) else 0,axis=1)

In [35]:
# TPH promedio según HH charge cell
TPH_historico=df_cost[["HH charge cell","TPH"]].groupby("HH charge cell").mean().reset_index()
TPH_historico

,HH charge cell,TPH
0,660.000000,4349.634608
1,660.002292,4295.012475
2,660.004585,4294.244128
3,660.006877,4304.960976
4,660.009169,4307.145630
...,...,...
2971,1012.710324,4058.039942
2972,1012.780870,4066.605017
2973,1012.851417,4029.256336
2974,1012.921963,4015.668759


In [36]:
def cost_TPH(df,cc):
    df["distance"]=(df["HH charge cell"]-cc).abs()
    x=df[df["distance"]==df.distance.min()]["TPH"].values[0]
    return x

In [37]:
df=df_cost[(df_cost["loss of TPH"]==0) & (df_cost["Rec HH CC less than HH CC"]==1) & (df_cost["greater than Rec HH CC"]==1)]
df.head(1)

,sag mill expert system,Limitado por Límite Alto de Tonelaje,Optimizando tonelaje,Limitado por Peso,Limitado por Corriente Bobina,Limitado por torque espesador 2,Limitado por torque espesador 3,Limitado por Potencia Bolas 1,Limitado por Potencia Bolas 2,Limitado por Pebbles,Limitado por Corriente harnero,Limitado por Potencia SAG,Limitado por limite bajo de tonelaje,Limitado por Temperatura bobina,Limitado por presión espesador 2,Limitado por presión espesador 3,pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,sag power index,ball work index,bornite law,torque,charge cell,power,TPH,solid percentage,water,speed,solid,LL charge cell,HH charge cell,HH TPH,stator current,granulometry,Edad,dif_TPH_HH,dif_CC_HH,dif_CC_LL,loss of TPH,cluster,recommendation,loss of TPH_1,loss of TPH_2,loss of TPH_3,loss of TPH_4,loss of TPH_5,greater than HH CC,greater than HH CC_1,greater than HH CC_2,greater than HH CC_3,greater than HH CC_4,greater than HH CC_5,less than LL CC,less than LL CC_1,less than LL CC_2,less than LL CC_3,less than LL CC_4,less than LL CC_5,greater than Rec HH CC,greater than Rec HH CC_1,greater than Rec HH CC_2,greater than Rec HH CC_3,greater than Rec HH CC_4,greater than Rec HH CC_5,Rec HH CC less than HH CC,Rec HH CC less than HH CC_1,Rec HH CC less than HH CC_2,Rec HH CC less than HH CC_3,Rec HH CC less than HH CC_4,Rec HH CC less than HH CC_5,start loss tph,end loss tph,loss of TPH_mod
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-10 06:57:00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.71,1.54,0.63,0.03,23.8025,113.3813,16.7595,0.0,66.54013,847.172356,22742.376417,4095.106281,74.626313,1397.367087,9.062103,74.164986,770.0,850.0,4100.0,3876.23877,45.912153,1.0,4.893719,2.827644,77.172356,0,0,847.066082,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,1,1.0,1.0,1.0,1.0,1.0,0,0,0


In [38]:
df_metric=df[["charge cell","TPH","HH TPH","recommendation","HH charge cell"]]
df_metric["TPH historic"]=df["recommendation"].apply(lambda x: cost_TPH(TPH_historico,x))
df_metric["cost TPH"]=df_metric["TPH"]-df_metric["TPH historic"]
df_metric

,charge cell,TPH,HH TPH,recommendation,HH charge cell,TPH historic,cost TPH
Timestamp,,,,,,,
2020-01-10 06:57:00,847.172356,4095.106281,4100.0,847.066082,850.0,2572.376270,1522.730011
2020-01-10 06:58:00,849.014070,4083.388754,4100.0,847.066082,850.0,2572.376270,1511.012484
2020-01-10 06:59:00,848.691768,4080.461824,4100.0,847.066082,850.0,2572.376270,1508.085554
2020-01-10 07:00:00,848.107939,4069.748964,4100.0,847.066082,850.0,2572.376270,1497.372695
2020-01-10 07:01:00,847.998278,4111.930940,4100.0,847.066082,850.0,2572.376270,1539.554670
...,...,...,...,...,...,...,...
2022-02-28 09:54:00,914.137054,4100.673792,4120.0,857.245190,950.0,3579.386529,521.287264
2022-02-28 09:55:00,919.262837,4090.384509,4120.0,857.245190,950.0,3579.386529,510.997980
2022-02-28 09:56:00,922.912778,4082.137827,4120.0,857.334396,950.0,3579.386529,502.751298


In [39]:
costo_TPH=df_metric["cost TPH"].sum()
costo_TPH

22631791.64528973

# TPH ganado!

In [40]:
tph1-tph2-costo_TPH

-23025865.65580122